In [ ]:
(ql:quickload "kenzo")

In [ ]:
(use-package "CAT")

In [ ]:
(cat-init)

# Simplicial Sets

## Introduction

A *simplicial set* (**Hilton &  Wylie**, *Homology theory*, Cambridge University Press, 1967) $K$ is a union $K={\displaystyle{\bigcup_{q\ge 0}{K^q}}}$, where the $K^q$ are disjoints sets, together with functions\index{simplicial sets}

$$\partial_i^q: K^q \longrightarrow K^{q-1},\quad q>0,\quad i=0,\ldots , q,$$

$$\eta_i^q: K^q \longrightarrow K^{q+1},\quad q\ge 0,\quad i=0,\ldots ,q,$$

subject to the relations

$$
\begin{align}
\partial_i^{q-1}\partial_j^q   & = & \partial_{j-1}^{q-1}\partial_i^q, &   i < j \cr
\eta_i^{q+1}\eta_j^q       & = & \eta_j^{q+1}\eta_{i-1}^q,    & i > j    \cr
\partial_i^{q+1}\eta_j^q     & = & \eta_{j-1}^{q-1}\partial_i^q,   & i < j  \cr
\partial_i^{q+1}\eta_i^q     & = & \partial_{i+1}^{q+1}\eta_i^q    & Identity,   \cr
\partial_i^{q+1}\eta_j^q     & = & \eta_j^{q-1}\partial_{i-1}^q,   &  i > j+1.\cr 
\end{align}
$$

An element of $K^q$ is an (*abstract*) $q$-simplex of $K$ and the functions $\partial$ and $\eta$ are respectively the *face operators* and the *degeneracy operators*. Formally, their action on a simplex is very simple. For instance, if a $q$--simplex is an ordered set of vertices, like $\lbrace v_0, v_1, \ldots,v_i,\ldots, v_q \rbrace$, the rules are the following (we omit the indication of the dimension of the simplex):

$$
\begin{eqnarray*}
\partial_i \lbrace v_0, v_1, \ldots,v_i,\ldots, v_q \rbrace & = &
           \lbrace v_0, v_1, \ldots,{\bf v_{i-1}}, {\bf v_{i+1}},\ldots, v_q \rbrace, \\
\eta_i \lbrace v_0, v_1, \ldots,v_i,\ldots, v_q \rbrace & = &
      \lbrace v_0, v_1, \ldots,{\bf v_i},{\bf v_i},\ldots, v_q \rbrace.
\end{eqnarray*}
$$

The operators $\partial_i^q$ will be used hereafter to define the boundary operator $d^q$ for the $q$-component of the associated chain complex:

$$d^q = \sum_{i=0}^q { (-1)^i \partial_i^q}.$$

The image of a simplex under some $\eta$ is called *degenerate*, because it
is not directly implied  in the realization of the simplicial set.

### Example

Let us take the small example *diabolo* from the chapter 1. For a better understanding, it is convenient to use a list representation for the simplices. So the set $K^0$ of 0-simplices (vertices) is:

$$\lbrace (0), (1),\ldots (5) \rbrace.$$

The set $K^1$ of 1-simplices contains the set of regular simplices:

$$\lbrace (0\  1), (0\  2), (1\  2), (2\  3), (3\  4), (3\  5), (4\  5) \rbrace,$$

but also the degenerate 1-simplices:

$$\lbrace (0\ 0), (1\ 1), (2\ 2),\ldots,(5 \ 5) \rbrace.$$

The set $K^2$ of 2-simplices contains the regular singleton:  

$$\lbrace ( 3\  4\  5) \rbrace$$

but also the degenerate 2-simplices:

$$\lbrace (0\ 0\ 0), \ldots, (5\ 5\ 5), (0\ 0\ 1), (0\ 0\  2), (1\ 1\ 2),\ldots, (4\ 5\ 5)\rbrace.$$

The sets $K^q$ ($q >2$) contain only degenerate simplices. With this  naming of the elements of a $q$-simplex, the action of $\partial$ and $\eta$ is now clear. For instance

$$\partial_0 (0\  1) = (1),\  \partial_1 (0\  1) = (0),$$

$$\eta_0 (0\  1) = (0\  0\  1),\ 
  \eta_1 (1\  2)= (1\  2\  2),\ \eta_2(3\ 4\ 5)=(3\ 4\ 5\ 5). $$

By the way, we see that a simplex is non-degenerate if the list is strictly increasing.

## Notion of abstract simplex in this Lisp implementation

In this implementation, on account of the essential following property: *any simplex
may be expressed in an unique way as a possibly iterated degeneracy of a non--degenerate simplex*, we call an **abstract simplex**, (in short **absm**), a pair consisting of:
- a (possibly iterated) degeneracy operator,
- a "geometric" simplex, i.e. a non--degenerate simplex.
For instance, if $\sigma$ is a non-degenerate simplex, and $\sigma'$ is the degenerate simplex  $\eta_3\eta_1\sigma$, the corresponding abstract simplices are respectively $\lbrace \emptyset, \sigma \rbrace$ and $\lbrace \eta_3\eta_1, \sigma \rbrace$.

An abstract simplex is represented internally in the system by the following lisp object:
$$\texttt{(:absm}\quad\mathit{dgop}\quad\texttt{.}\quad\mathit{gmsm}\texttt{)}$$
where,
1. *dgop* is a non-negative integer coding a *strictly decreasing integer list*. This strictly  decreasing list of integers represents a sequence of $\eta$ operators and is coded as a **unique** integer with the following convention: a number $i$ in the sequence is the $i+1$--th binary bit of a machine word. The null list is coded as the number 0 whereas the list ``(0)``, i.e. the $\eta_0$ operator, is coded as the integer ``1``, etc. Two functions are provided to help the user for the transformation in both directions.
2. *gmsm* is a geometric simplex,  i.e. any kind lisp object modelizing a non-degenerate simplex. For practical reason, a  type ``GMSM``, (any kind of lisp object), has been defined.

The simplest constructor is the macro ``(absm (dgop-ext-int`` *dgop-list*``)``  *gmsm*``)``, where *dgop-list* is a strictly decreasing integer list. A type ``ABSM`` has been defined. The associated printing method prints such an object under the form:
$$\texttt{<AbSm}\quad\mathit{ext-dgop}\quad\mathit{gmsm}{\texttt >}$$
where *ext-dgop* shows clearly the sequence of the operators $\eta_i$ (see the examples). The accessor functions for the components are the macros ``dgop`` and ``gmsm``.

### Simple functions for abstract simplices

``dgop-ext-int`` *ext-dgop* *[Function]*
>Code on an integer the valid list representing the degeneracy operator *ext-dgop*.

``dgop-int-ext`` *dgop* *[Function]*
> Give the list representing a degeneracy operator from the integer *dgop*.

``absm`` *dgop gmsm* *[Macro]*
> Create an abstract simplex, using directly the coded representation, *dgop*, of the degeneracy operator.

``dgop`` *absm* *[Macro]*
> Get the integer code of the internal representation of the degeneracy operator of the abstract simplex *absm*.
 
``gmsm`` *absm* *[Macro]*
> Get the basic non-degenerate simplex part of the abstract simplex ``absm``.

``absm-p`` *object* *[Predicate]*
>Test if *object* is an abstract simplex.

### Examples

Let us suppose that ``:sigma`` "is" a geometric simplex, we may construct the abstract simplices of the beginning of the section.

In [ ]:
(dgop-ext-int '(0))

In [ ]:
(dgop-ext-int '(4 0))

In [ ]:
(dgop-int-ext 63)

In [ ]:
(dgop-ext-int '(2 2))

In [ ]:
(def asigma (absm 0 :sigma))

In [ ]:
(def asigma-prime (absm (dgop-ext-int '(3 1)) :sigma))

In [ ]:
(absm-p asigma-prime)

The following  command gives the sixth degeneracy of the vertex 0 represented as ``(0)``.

In [ ]:
(def deg-6 (absm (dgop-ext-int '(5 4 3 2 1 0)) '(0)))

In [ ]:
(dgop deg-6)

In [ ]:
(gmsm deg-6)

## Representation of a simplicial set

A simplicial set is implemented as an instance of the class ``SIMPLICIAL-SET``, subclass of the class ``COALGEBRA``.

```lisp
(DEFCLASS SIMPLICIAL-SET (coalgebra)
    ((face :type face :initarg :face :reader face1)))

```

This class inherits also from the class ``CHAIN-COMPLEX`` and has one slot of its own:

>``face``, a lisp function  computing any face of any geometric simplex of the simplicial set.
This is a function with 3 arguments: a face index (a non--negative integer, ``indx``), a simplex dimension (a positive integer, ``dmns``) and a geometric simplex (``gmsm``), the $indx$-th face of which is looked for. Whatever the face simplex is, degenerate or not, the simplex returned by this function must be an **abstract** simplex, i.e. an ``ABSM`` object.

A printing method has been associated to the class ``SIMPLICIAL-SET`` and the external representation of  an instance is a string like ``[K``*n* ``Simplicial-Set]``, where $n$ is the number plate of the Kenzo object.

## The function ``build-smst``

To facilitate the construction of instances of the class ``SIMPLICIAL-SET``, the software ``Kenzo`` provides the function ``build-smst`` defined with keyword parameters:

``build-smst``
>``:cmpr`` *cmpr* ``:basis`` *basis* ``:bspn`` *bspn* ``:face`` *face*  
``:face*`` *face$*$* ``:intr-bndr`` *intr-bndr* ``:bndr-strt`` *bndr-strt*  
``:intr-dgnl`` *intr-dgnl* ``:dgnl-strt`` *dgnl-strt* ``:orgn`` *orgn*

The returned value is an object of type ``SIMPLICIAL-SET``. This function assigns an identification number ``idnm`` to the Kenzo object instance in a sequential way and pushes this object on the list of already  created simplicial sets instances, ``*smst-list*``.

The keywords parameters are:
- *cmpr*, a comparison function for generators. 
- *basis*, if the simplicial set is effective, this argument must be the lisp function returning the  list of non-degenerate simplices  of the simplicial set, in a given dimension; if the simplicial set is locally effective, this argument must be the keyword ``:locally-effective``.
- *bspn*, the lisp representation of the base point (an item of type ``GMSM``). This is in fact the slot ``bsgn`` of the associated chain complex.
- *face*, a lisp function for the face operators.
- $face*$, a lisp function for the face operators, returning the symbol ``:degenerate`` if the corresponding face is degenerate.
- *intr-bndr*, a lisp function for the differential of the associated chain complex.
- *bndr-strt*, the strategy (``:cmbn`` or ``:gnrt``) for the function *intr-bndr*.
- *intr-dgnl*, a lisp function for the coproduct of the coalgebra.
- *dngl-strt*, the strategy (``:cmbn`` or ``:gnrt``) for the function *intr-dgnl*.
- *org*, the comment list, adequately chosen.

The three arguments $face*$,  *intr-bndr* and *intr-dgnl* are not mandatory. Only the function *face* is necessary to define precisely the simplicial set. From the function *face*, we know that we may construct:
- The differential for the associated chain complex, according to the formula:
$$d^q = \sum_{i=0}^q { (-1)^i \partial_i^q}.$$
- The coproduct of the underlying coalgebra (the Alexander-Whitney diagonal, $\Delta$) according to the formula:
$$\Delta(\sigma)= \sum_{i=0}^{n}{{\tilde{\delta}}^{(i)}\sigma \otimes \delta_0^{(i)} \sigma},$$
where the power $(i)$ denotes the $i$-th composition of the operator and $\tilde{\delta}$ is the
*last face* operator, so that
$${\tilde{\delta}}^{(i)}=\delta_{n-i+1}\circ \cdots \circ \delta_{n-1}\circ \delta_n.$$

But, as the differential ignore degenerate faces, it may be interesting for the user to provide a face function, weaker than the function *face* but sufficient and more efficient to compute the boundary of a simplex. It is the raison d'etre of the parameter $face*$ which may be used as an alternative to *face*. In both cases, the strategy is set to ``:gnrt`` by the system. It may also arrive that the user knows a particularly simple form for the differential, he may then use the parameter *intr-bndr*. In the computation of differentials in the associated chain complex, this function will be used instead of the canonical differential computed from the face function. In this last case, the user is required to provide the strategy. Same remark for the parameter *intr-dgnl*.

### Examples

As first example, we give the construction of the simplicial set $\Delta^n$  corresponding to the 
standard $n$-simplex. This is given only as example. The program ``Kenzo`` provides a function
for the same purpose which will be described later. For every simplex, we shall use a list representation as in our first example ``diabolo``. The various keyword arguments for the call to ``build-smst``, are:
1. *bspn*, the list ``(0)``, i.e. the representation of the vertex 0.
2. *cmpr*, the lisp function  ``#'l-cmpr``, since this function is adequate to compare the lists representing the simplices.
3. *basis*, the lisp function returning the list of all possible non-degenerate simplices in some dimension. Here, this is the combinatory function returning the set of all the lists of $p+1$ objects taken among $n+1$. Such a function is for instance the function  (``delta-inj`` *p n*) whose definition is given hereafter. So, the ``:basis`` keyword argument is simply:
```lisp
#'(lambda(dmn) (delta-inj dmn n))
```.
Note that here, ``n`` is a global parameter.
4. *face*, the lisp function for the face operators (``face-delta-n``). It consists simply in removing the $i$-th element of the list describing a simplex ($i$ starting at 0).
5. *org* will be a comment.

In [ ]:
(defun delta-inj (m n)
  (declare (type fixnum m n))
  (cond ((> m n) +empty-list+)
        ((zerop m) (mapcar #'list (<a-b> 0 n)))
        (t (mapcan #'(lambda (list)
                       (declare (type list list))
                       (mapcar #'(lambda (k)
                                   (declare (type fixnum k))
                                   (cons k list))
                                 (<a-b< 0 (first list))))
                     (delta-inj (1- m) n)))))

In [ ]:
(delta-inj 0 3)

In [ ]:
(delta-inj 1 3)

In [ ]:
(delta-inj 3 3)

(def face-delta-n
  #'(lambda(i dmn gsm)
      (declare(ignore dmn))
      (absm 0 (append (subseq gsm 0 i)
                      (subseq gsm (1+ i))) )))

To create $\Delta^n$ for any order $n$, we embed the call of ``build-smst`` in a function having $n$ as parameter.

In [ ]:
(defun delta-n(n)
  (declare (type fixnum n))
        (build-smst :cmpr #'l-cmpr
                    :basis #'(lambda(dmn)(delta-inj dmn n))
                    :bspn '(0)
                    :face face-delta-n
                    :orgn `(delta-n ,n)))

In [ ]:
(def delta4 (delta-n 4))

In [ ]:
(bspn delta4)

In [ ]:
(basis delta4 0)

In [ ]:
(basis delta4 3)

A second example is the contruction of the simplicial complex  $\Delta^\mathbb{N}$ freely generated by the positive integers. The construction of the ``SIMPLICIAL-SET`` instance to implement this simplicial set is quite similar to that of $\Delta^n$, but in this case the ``:basis`` keyword argument is  the keyword ``:locally-effective``, since the sets $K^q$ are infinite. Whence, the definition of $\Delta^\mathbb{N}$:

In [ ]:
(def delta-infty
        (build-smst :cmpr #'l-cmpr
                    :basis :locally-effective
                    :bspn '(0)
                    :face face-delta-n
                    :orgn `(delta-infinity)))

A third example is the construction of the simplicial set corresponding to ``diabolo``. This simplicial set is a  sub-complex of $\Delta^2$. The ``:face`` function is exactly the same as in the examples above and the ``:basis`` function returns the sub-simplices of ``diabolo``, by enumeration. 

In [ ]:
(def diabolo-ss 
        (build-smst :cmpr #'l-cmpr
                    :basis  #'(lambda (dmn)
                                (case dmn
                                      (0 '((0)(1)(2)(3)(4)(5)))
                                      (1 '((0 1)(0 2)(1 2)(2 3)(3 4)(3 5)(4 5)))
                                      (2 '((3 4 5)))))
                    :bspn '(0)
                    :face face-delta-n
                    :orgn '(simplicial set for diabolo)))

## A first set of helpful functions on simplicial sets

We give now a first set of functions, methods or macros acting on  simplicial set instances. 

``cat-init`` *[Function]*
> Clear in particular ``*smst-list*``, the list of user created simplicial sets  and reset the global counter to 1.

``smst`` *n* *[Function]*
> Retrieve in the list ``*smst-list*`` the simplicial set instance whose the Kenzo identification
is $n$. If it does not exist, return ``NIL``.

``cmpr`` *smst absm1 absm2* *[Macro]*
> Compare the abstract simplices *absm1* and *absm2* with the comparison function associated with the simplicial set *smst*.

``bspn`` *smst* *[Macro]*
> Return the base point of the simplicial set *smst*.

``bndr``  *smst* ``&rest`` *[Macro]*
> Identical to the macro ``dffr`` (see the chain complex chapter). For simplicial sets, it is more traditional to use the term *boundary*.

``dgnl`` *smst* ``&rest`` *[Macro]*
> Identical to  the macro ``cprd`` (see the coalgebra chapter). For simplicial sets, this recalls the 
Alexander-Witney diagonal.

``face`` *smst i k  gmsm-or-absm* *[Macro]*
> Versatile macro to apply the face operator $\partial_i^k$ on the simplex *gmsm-or-absm* of the simplicial set *smst*. The name of the simplex parameter shows clearly that it may be either a geometric simplex or an abstract simplex. With only one argument (*smst*), return the function *face*. 
In any other case, we recall that this application   returns always  an object of type ``ABSM``.

``degenerate-p`` *absm* *[Macro]*
> Return ``t`` if the abstract simplex *asm* is degenerate, ``nil`` otherwise.

``non-degenerate-p`` *absm* *[Macro]*
> Return ``t`` if the abstract simplex *asm* is non-degenerate, ``nil`` otherwise.

``check-smst`` *smst dmns1* ``&optional`` *dmns2 (1+ dmns1)* *[Function]*
> Return ``t`` if the fundamental relations between the face operators $\partial_i^k$
with $k=dmns1,\ldots, dmns2-1$, applied on the effective simplicial set *smst*, are satisfied, otherwise ``nil``. If the optional parameter *dmns2* is omitted, the test is done only for
$k=dmns1$.

## Examples

After having created instances of the standard simplicial sets $\Delta^2$ and $\Delta^3$ ($\Delta^\mathbb{N}$ has been defined previously), let us test some simple functions. Note that, since the class of simplicial sets is a sub-class of the class of coalgebras, itself subclass of chain complexes, the functions defined on these classes are available.

In [ ]:
(def delta2 (delta-n 2))

In [ ]:
(def delta3 (delta-n 3))

In [ ]:
(degenerate-p (absm 0 '(1 2 3)))

In [ ]:
(degenerate-p (absm 5 '(0 1 2 3 4)))

In [ ]:
(basis delta3 2)

In [ ]:
(basis delta2 0)

  The set of simplices in any dimension for the simplicial set $\Delta^\mathbb{N}$ is infinite:

In [ ]:
(basis delta-infty 1)

Let us test the face function. We see that, though ``delta-infty`` is locally effective, nevertheless, we may work with its simplices:

In [ ]:
(face delta-infty 2 4 '(0 1 2 3 4))

In [ ]:
(face delta-infty 2 4 (absm 0 '(0 1 2 3 4)))

In [ ]:
(face delta-infty 2 5 (absm (dgop-ext-int '(1)) '(0 1 2 3 4)))

In [ ]:
(face delta-infty 2 5 (absm (dgop-ext-int '(3)) '(0 1 2 3 4)))

In [ ]:
(bspn delta-infty)

The two following statement show that ``delta-infty`` may be considered also as a coalgebra and a chain complex.

In [ ]:
(cprd delta-infty 4 '(0 1 2 3 4))

In [ ]:
(? delta-infty 4 '(0 1 2 3 4))

The simplicial set ``diabolo-ss``,  has also all the properties of a coalgebra and of a chain complex.

In [ ]:
(basis diabolo-ss 1)

In [ ]:
(basis diabolo-ss 2)

In [ ]:
(? diabolo-ss 2 (first *))

In [ ]:
(? diabolo-ss *)

In [ ]:
(dgnl diabolo-ss **)

## The function ``build-finite-ss``

The function ``build-finite-ss`` is a powerful function simplifying the creation of a finite simplicial set. From the lisp point of view, it accepts only one argument, the structured list of the simplices and their faces. In this list, every non--degenerate simplex must be coded as a symbol given by the user.  The  structure of the list is the following:

- **0**    (in fact, optional)
- $<$Sequence of the vertices, the first item being the base point$>$,
- **1**
- $<$Sequence of description of simplices in dimension 1$>$,
- **2**
- $<$Sequence of description of simplices in dimension 2$>$,
-   ... ... ... ... ... ...
- **n**
- $<$Sequence of description of simplices in dimension n$>$.

where $<$Sequence of description of simplices in dimension k$>$ is a sequence of the following items:

1. A symbol, the symbolic name of the simplex.
2. A list of  faces of this simplex, describing the faces of the simplex. A face can be described in two ways:
  - a symbol, the symbolic name of a {\bf non-degenerate} simplex composing itself the face.
  - or a list of the form  ($i_k\ i_{k-1}\ \ldots \ i_1 \ i_0$ ``symbol``) meaning that the degeneracy $\eta_{i_k}\eta_{i_{k-1}}\ldots\eta_{i_1}\eta_{i_0}$ is to be applied to the non-degenerate simplex of name ``symbol``  to produce the actual (possibly degenerate) face. If the simplex face has dimension 0 (a vertex), the name of this vertex is sufficient in the face description because the program computes itself the unique possible degeneracy of the vertex, i.e. $\eta_{k-2}\ldots\eta_0$. 

### Examples

Let us give some examples to clarify the use of ``build-finite-ss``. In the first example, we use  ``build-finite-ss`` to create a simplicial complex, corresponding to a triangulation of the torus. As the faces are never degenerated, the description of the faces is of simplified form.

In [ ]:
(cl-jupyter-user:png-from-file "torus2.png")

The vertices will be named ``v0``, ``v1``, ..., ``v6``, ``v0`` being the base point. The edges
will be named ``e01``, ``e02``, ...,  ``e56``, the list of faces of the edge $e_{ij}$ being
$(v_j\  v_i)$. The triangles will be named ``t013``, ``t015``, ..., ``t456``, the list of faces of
the triangle $t_{ijk}$ being $(e_{jk}\  e_{ik}\  e_{ij})$. With this convention of describing the simplices, the call to ``build-finite-ss`` is in simplified form:

In [ ]:
(def torus2
    (build-finite-ss 
       '(v0  v1  v2  v3  v4  v5  v6 

         1 e01  (v1 v0)   e02  (v2 v0)   e03  (v3 v0)
           e04  (v4 v0)   e05  (v5 v0)   e06  (v6 v0)
           e12  (v2 v1)   e13  (v3 v1)   e14  (v4 v1)
           e15  (v5 v1)   e16  (v6 v1)   e23  (v3 v2)
           e24  (v4 v2)   e25  (v5 v2)   e26  (v6 v2)
           e34  (v4 v3)   e35  (v5 v3)   e36  (v6 v3)
           e45  (v5 v4)   e46  (v6 v4)   e56  (v6 v5)

         2 t013  (e13 e03 e01)  t015  (e15 e05 e01)  t024  (e24 e04 e02)
           t026  (e26 e06 e02)  t036  (e36 e06 e03)  t045  (e45 e05 e04)
           t125  (e25 e15 e12)  t126  (e26 e16 e12)  t134  (e34 e14 e13)
           t146  (e46 e16 e14)  t234  (e34 e24 e23)  t235  (e35 e25 e23)
           t356  (e56 e36 e35)  t456  (e56 e46 e45)  )))

After verification of the coherence of the description of the faces, the function ``build-finite-ss`` calls adequately the function ``build-smst`` to create an instance of the class ``SIMPLICIAL SET`` subclass of the class ``CHAIN COMPLEX``. So that, it is easy for the user  to obtain the homology groups of the torus, for instance in dimension 1:

In [ ]:
(chcm-homology  torus2 1)

The function ``build-finite-ss`` calls internally some verification functions mentioned above to verify the coherence of the description of the simplicial set and gives an error message if the description of the simplicial set is incorrect. For instance:

In [ ]:
(def mm (build-finite-ss 
   '(s0 s1 s2 s3 
     1 0-1 (s1 s0)  1-2 (s2 s1)  2-3 (s3 s2)
     2 0-1-2 (2-3 1-2 0-1))
  ))

In the following examples, we show the power of ``build-finite-ss`` for the description of classical surfaces as simplicial sets, and in each case we  verify the well known homology groups of those surfaces. The examples of the torus or the dunce-hat must be compared, as to the simplicity of the definition, to the respective ones in the previous page and in the Homology chapter.

In [ ]:
(cl-jupyter-user:png-from-file "cylinss.png")

In [ ]:
(def cylinss (build-finite-ss 
    '(v0 v1 
      1 e0 (v1 v0) e1 (v1 v1) e2 (v0 v0) e3 (v1 v0)
      2 t0 (e1 e3 e0) t1 (e0 e3 e2)) ))

In [ ]:
(dotimes (i 3) (chcm-homology cylinss i))

In [ ]:
(cl-jupyter-user:png-from-file "moebiuss.png")

In [ ]:
(def moebiuss (build-finite-ss 
  '(v0 v1 
    1 e0 (v1 v0) e1 (v0 v1) e2 (v1 v0) e3 (v0 v0)
    2 t0 (e1 e3 e0) t1 (e0 e2 e3)) ))

In [ ]:
(dotimes (i 3) (chcm-homology moebiuss i))

In [ ]:
(cl-jupyter-user:png-from-file "toruss.png")

In [ ]:
(def toruss (build-finite-ss 
  '(v0
    1 e0 (v0 v0) e1 (v0 v0) e2 (v0 v0)
    2 t0 (e1 e2 e0) t1 (e0 e2 e1)) ))

In [ ]:
(dotimes (i 3) (chcm-homology toruss i))

In [ ]:
(cl-jupyter-user:png-from-file "bottless.png")

In [ ]:
(def bottless (build-finite-ss 
  '(v0
    1 e0 (v0 v0) e1 (v0 v0) e2 (v0 v0)
    2 t0 (e1 e2 e0) t1 (e0 e1 e2)) ))

In [ ]:
(dotimes (i 3) (chcm-homology bottless i))

In [ ]:
(cl-jupyter-user:png-from-file "ppss.png")

In [ ]:
(def ppss (build-finite-ss 
  '( *
     1 e (* *)
     2 t (e * e)) ))

In [ ]:
(dotimes (i 3) (chcm-homology ppss i))

The user will note that the list of faces for the 2-simplex ``t``, is in simplified form. In particular, as the face 1 of ``t`` is the 0-degeneracy of the base point "$*$", it is sufficient to code the  face 1 of ``t``by "``*``" instead of the complete form ``(0 *)``.

In [ ]:
(cl-jupyter-user:png-from-file "duncess.png")

In [ ]:
(def duncess (build-finite-ss 
  '( *
     1 e (* *)
     2 t (e e e)) ))

In [ ]:
(dotimes (i 3) (chcm-homology duncess i))

## Special simplicial sets

The system provides useful functions to create interesting simplicial sets of constant usage. In particular, the user is advised from now to use the following version of the standard simplex implemented in ``Kenzo`` and not the one given previously for simplicity reason.

### The standard simplex

``delta`` *dmns* *[Function]*
> Create the standard simplex $\Delta^n$ with vertices $0,1,\ldots,n$, where *dmns* is the parameter for $n$. This simplicial set is so important in the applications that the implementor has decided to code the simplices in a  way similar to the coding
of the degeneracy operators. An increasing sequence of non-negative integers describing a simplex, is coded on a binary integer with the following convention: a number $i$ representing the vertex $i$ is the $(i+1)$--th binary bit of a machine word. The base point $0$ is the bit 1 in position 1 of the word. This representation is very efficient for saving memory space but somehow awkward to read.

``delta-infinity`` *[Function]*
> Create the locally effective standard simplex $\Delta^\mathbb{N}$ freely generated by the positive integers.

``deltab`` *[Function]*
> Create the locally effective *reduced* simplicial set $\bar \Delta$, built from $\Delta^\mathbb{N}$ by identifying all the vertices to the base point.

``deltab2`` *[Function]*
> Create the locally effective *1-reduced* simplicial set $\Delta_2^\mathbb{N}$, obtained from the above $\bar \Delta$, by identification of all the edges with the base point. This is the efficient version of the coalgebra that is used as example in the cobar chapter.

``dlop-ext-int`` *ext-dlop* *[Function]*
> Code on an integer the valid list (increasing order) representing the simplex *ext-dlop* of the standard simplex.

``dlop-int-ext`` *dgop* *[Function]*
> Give the list representing a simplex of the standard simplex from the integer *dgop*.

``vertex-i`` *absm i* *[Function]*
> Give the $i$-th vertex of the (coded, degenerate or not) abstract simplex ``absm`` belonging to a simplicial set of the $\Delta$ family.

``absm-ext-int`` *vlist* *[Function]*
> Create a valid abstract simplex of the $\Delta$ family from the simplex *vlist* written as a non-decreasing list of non-negative integers (i.e coding of any degenerate or non--degenerate simplex of the $\Delta$ family).

``absm-int-ext`` *absm* *[Function]*
> From an internal coded form of an abstract simplex of the $\Delta$ family, create the  non-decreasing list
of non--negative integers representing the canonical external form of such a simplex.

``soft-delta`` *dmns* *[Function]*
> Create another version of the standard simplex $\Delta^n$ designed for a better clarity in the printing of the results at testing time. More precisely, the simplices are represented internally by a list of the form ``(:delt`` *binary-code*``)``. With this representation, the system is able to recognize a coded simplex and to print it in a readable form. For the user, the only
requirement is to write a simplex coded $n$ as ``(d`` *n*``)``, where ``d`` is the macro building the internal representation (see the examples). Attention: the above conversion functions do not work with this representation.

``soft-delta-infinity`` *[Function]*

### Examples

In [ ]:
(def d3 (delta 3))

The simplices $2$, $4$ and  $8$ are the coded representation of the vertices $1$, $2$ and $3$:

In [ ]:
(cmpr d3 2 4)

In [ ]:
(cmpr d3 4 4)

In [ ]:
(cmpr d3 8 4)

The list returned by the following statement  getting the basis of $\Delta^3$ in dimension 2, namely ``(7 11 13 14)`` is in fact the coded list of the following simplices ``((0 1 2) (0 1 3) (0 2 3) (1 2 3))``.

In [ ]:
(basis d3 2)

In the following statement, the integer $21$ represents the simplex in dimension $2$, ``(0 1 4)``, $1$ represents the operator $\partial_1$ and the face is therefore ``(0 4)`` represented as the integer $17$.

In [ ]:
(face d3 1 2 21)

We may obtain the same result, using:

In [ ]:
(face d3 1 2 (dlop-ext-int '(0 1 4)))

Let us test the conversion functions.

In [ ]:
(vertex-i (absm 0 1) 0)

In [ ]:
(vertex-i (absm 1 1) 0)

In [ ]:
(vertex-i (absm 1 1) 1)

In [ ]:
(vertex-i (absm 0 7) 2)

In [ ]:
(absm-ext-int '(0 0 0 1 2 3 3 3))

In [ ]:
(absm-ext-int '(0 1 1 1 2))

In [ ]:
(absm-int-ext (absm-ext-int '(0 0 0 1 2 3 3 3)))

The following  call to the macro ``?`` returns the boundary of the simplex ``(0 2 3)`` of $\Delta^3$, the integers $5$, $9$ and $12$ representing respectively the simplices ``(0 2)``, ``(0 3)`` and ``(2 3)``. The application of the coproduct ``dgnl`` to the simplex ``(0 1 2 3)``, coded $15$, is also easily interpreted:

In [ ]:
(? d3 2 13)

In [ ]:
(dgnl d3 3 15)

We may show now some examples with the *soft* version of the standard simplex. Using the macro ``d`` and the function ``dlop-ext-int`` the user may work with a readable form of the simplices and the degeneracy operators.

In [ ]:
(def d3 (soft-delta 3))

In [ ]:
(cmpr d3 (d 2) (d 4))

In [ ]:
(basis d3 1)

In [ ]:
(dgnl d3 3 (d (dlop-ext-int '(0 1 2 3))))

In [ ]:
(face d3 1 2 (d (dlop-ext-int '(0 2 4))))

In [ ]:
(? d3 2 (d (dlop-ext-int '(0 2 3))))

### Spheres, Moore spaces and projectives spaces